In [36]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")



In [45]:
collection_info = client.get_collection(collection_name="words")

print(f"Số lượng vectors trong collection 'words': {collection_info.points_count}")


Số lượng vectors trong collection 'words': 1585650


In [38]:
print(collection_info.config)


params=CollectionParams(vectors=VectorParams(size=100, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None) hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None) optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None) wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0) quantization_config=None


In [29]:
from qdrant_client import QdrantClient, models

# Kết nối tới Qdrant server

# Tạo collection với cấu hình HNSW
client.create_collection(
    collection_name="words",
    vectors_config=models.VectorParams(
        size=100,  # Số chiều của vector
        distance=models.Distance.COSINE  # Khoảng cách Cosine
    ),
    hnsw_config={  # Sử dụng dictionary thay vì đối tượng HnswConfig
        "m": 16,  # Giữ mức kết nối vừa phải
        "ef_construct": 100,  # Tối ưu trong quá trình xây dựng chỉ mục
        "full_scan_threshold": 10000  # Ngưỡng cho full scan
    }
)


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `words` already exists!"},"time":0.000934331}'

In [48]:
from qdrant_client import QdrantClient, models

# Kết nối tới Qdrant server
# client = QdrantClient(url="http://localhost:6333")

# Bước 1: Sử dụng scroll để tìm kiếm từ "tội_phạm" dựa trên payload index
scroll_result, _ = client.scroll(
    collection_name="words",
    limit=1,  # Chỉ cần tìm 1 kết quả (vector của "tội_phạm")
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="word",  # Tên của trường payload
                match=models.MatchValue(value="thầy_giáo")  # Tìm kiếm theo từ "tội_phạm"
            )
        ]
    ),
    # with_vectors=True  # Trả về cả vector trong kết quả
)

# Kiểm tra xem từ "tội_phạm" có tồn tại không
if scroll_result:
    # Lấy vector của từ "tội_phạm"
    toi_pham_vector = scroll_result[0].vector

    # Bước 2: Tìm các từ có vector gần nhất với "tội_phạm"
    similar_words = client.search(
        collection_name="words",
        query_vector=toi_pham_vector,  # Vector của "tội_phạm"
        limit=10  # Tìm top 10 từ tương đồng nhất
    )

    # Hiển thị kết quả tìm kiếm
    for result in similar_words:
        print(f"ID: {result.id}, Word: {result.payload['word']}, Score: {result.score}")
else:
    print("Không tìm thấy từ 'tội_phạm' trong collection.")


ID: 1043, Word: tội_phạm, Score: 1.0
ID: 15431, Word: Tội_phạm, Score: 0.8421682
ID: 6735, Word: băng_nhóm, Score: 0.81849104
ID: 706, Word: ma_tuý, Score: 0.8084944
ID: 194, Word: đối_tượng, Score: 0.7743817
ID: 10384, Word: băng_đảng, Score: 0.7652103
ID: 10064, Word: phạm_pháp, Score: 0.75980574
ID: 1520, Word: hình_sự, Score: 0.7513312
ID: 2158, Word: phạm_tội, Score: 0.74307966
ID: 3539, Word: buôn_lậu, Score: 0.7291214


In [1]:
from qdrant_client import QdrantClient

# Kết nối tới Qdrant server
client = QdrantClient(url="http://localhost:6333")

# Xóa collection "words"
client.delete_collection(collection_name="words")


True